# Nequi project

### 1. Based on the link provided, files are dowloaded with aws cli.

In [1]:
!aws s3 cp "s3://nequi-open-data/sample_data_0006_part_00.parquet" .

download: s3://nequi-open-data/sample_data_0006_part_00.parquet to ./sample_data_0006_part_00.parquet


In [2]:
!aws s3 cp "s3://nequi-open-data/sample_data_0007_part_00.parquet" .

download: s3://nequi-open-data/sample_data_0007_part_00.parquet to ./sample_data_0007_part_00.parquet


### 2. Reading the file

#### 2.a Pandas is first used to read parquet files

In [1]:
import pandas as pd

In [2]:
sample006 = pd.read_parquet('./sample_data_0006_part_00.parquet')
sample007 = pd.read_parquet('./sample_data_0007_part_00.parquet')

samples = pd.concat([sample006,sample007], ignore_index=True) # Since the whole data is stuided, samples are concatanated.

With Pandas it took 45.0 seconds to read and concatenate the files.

#### 2.b Pyspark is used to improve times

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder.appName("NequiProject").getOrCreate()

23/08/08 21:13:28 WARN Utils: Your hostname, Danielas-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.8 instead (on interface en0)
23/08/08 21:13:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/08 21:13:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
sample006 = spark.read.parquet('./sample_data_0006_part_00.parquet')
sample007 = spark.read.parquet('./sample_data_0007_part_00.parquet')
samples = sample006.union(sample007)

Since pyspark is more time efficient, the project will use it instead of pandas.

In [4]:
samples.show(5)

+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+------------------+----------------+
|         merchant_id|                 _id|          subsidiary|   transaction_date|      account_number|             user_id|transaction_amount|transaction_type|
+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+------------------+----------------+
|075d178871d8d4850...|aa8dacff663072244...|824b2af470cbe6a65...|2021-09-12 13:32:03|648e257c9d74909a1...|ba42d192a145583ba...|      178.33365037|         CREDITO|
|075d178871d8d4850...|a53bb81bd0bba2ae2...|2d8d34be7509a6b12...|2021-09-12 13:31:58|c0b62f9046c83ea55...|5cfff960ea6d732c1...|       35.66673007|         CREDITO|
|075d178871d8d4850...|79f893ea65c06fe29...|5eeb18254850b21af...|2021-09-12 13:31:56|872d10143fc0ac7d5...|c97e63a92c82c7217...|      142.66692029|         CREDITO|
|075d178871d8d4850...|

In [5]:
samples.printSchema()

root
 |-- merchant_id: string (nullable = true)
 |-- _id: string (nullable = true)
 |-- subsidiary: string (nullable = true)
 |-- transaction_date: timestamp (nullable = true)
 |-- account_number: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- transaction_amount: decimal(24,8) (nullable = true)
 |-- transaction_type: string (nullable = true)



In [6]:
samples.select("user_id").show(5, truncate=False)

+--------------------------------+
|user_id                         |
+--------------------------------+
|ba42d192a145583ba8e7bf04875f837f|
|5cfff960ea6d732c1ba3e63d24f3be52|
|c97e63a92c82c7217b333635d75928ed|
|fc09bdd00f283222d65eaff4d00a6594|
|213527e8ba94fcaf2f9378969f9f6abc|
+--------------------------------+
only showing top 5 rows



In [7]:
samples.select(F.year("transaction_date").alias("Year")).distinct().show()

+----+
|Year|
+----+
|2021|
|2020|
+----+



In [8]:
samples.select("merchant_id").distinct().show(truncate=False)

+--------------------------------+
|merchant_id                     |
+--------------------------------+
|075d178871d8d48502bf1f54887e52fe|
|817d18cd3c31e40e9bff0566baae7758|
|838a8fa992a4aa2fb5a0cf8b15b63755|
+--------------------------------+



In [9]:
num_users = samples.select("user_id").distinct().count()
num_accounts = samples.select("account_number").distinct().count()

print(f"Total unique users that has placed at least one transaction are {num_users:,}, while there are a total of {num_accounts:,} accounts.")

Total unique users that has placed at least one transaction are 3,087,217, while there are a total of 3,099,711 accounts.


In [10]:
accounts = samples.groupBy("user_id").agg(F.countDistinct("account_number").alias("account_count")).orderBy(F.desc("account_count")).limit(20)
ammounts = accounts.join(samples, "user_id", "inner") \
    .groupBy("user_id") \
    .agg(F.sum("transaction_amount").alias("sum_transactions"),
         F.stddev("transaction_amount").alias("std_deviation"))
result = accounts.join(ammounts, 'user_id', 'inner').orderBy(F.desc("account_count")).show(truncate=False)

23/08/08 20:46:21 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/08/08 20:46:21 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/08/08 20:46:21 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/08/08 20:46:21 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/08/08 20:46:21 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/08/08 20:46:21 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/08/08 20:46:21 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/08/08 20:46:21 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/08/08 20:46:24 WARN RowBasedKeyValueBatch: Calling spill() on

+--------------------------------+-------------+----------------+------------------+
|user_id                         |account_count|sum_transactions|std_deviation     |
+--------------------------------+-------------+----------------+------------------+
|8c292781ac3e591312d7fc5b767687ca|43           |57794.36941102  |297.1821683130902 |
|6ba67792fbc4e375fa69ed6df0e44854|27           |59241.24976144  |441.69874871545954|
|328550eea11d2441ab258a1f07581dc8|24           |6242.86665343   |91.07442149253173 |
|a9fbdb2d32424dbac15edc0b17c12e23|23           |17405.36427600  |161.0434881880177 |
|62e1cb0b55d1d7b577292916528ef6d8|20           |1190.07989329   |23.55723555824086 |
|16100156a44d52a0ab40b2661dd648cc|17           |5350.00951085   |459.00474438452113|
|94cf8a83c7f7d2b2d33450d9ca233ef6|17           |15539.99429289  |374.8176078533819 |
|a71f0b8fc6267a4687430bea42310468|16           |13470.13505786  |249.89727904121185|
|e066a19e39bbb74e82d0d88702292b7e|13           |4761.50846469   |

In [11]:
samples.select("transaction_type").distinct().show(truncate=False)

+----------------+
|transaction_type|
+----------------+
|DEBITO          |
|CREDITO         |
+----------------+



In [12]:
samples.groupBy("merchant_id", F.year("transaction_date").alias("transaction_year")).agg(F.sum("transaction_amount").alias("total_transactions")).show(truncate=False)

+--------------------------------+----------------+-------------------+
|merchant_id                     |transaction_year|total_transactions |
+--------------------------------+----------------+-------------------+
|075d178871d8d48502bf1f54887e52fe|2021            |550735204.98759165 |
|817d18cd3c31e40e9bff0566baae7758|2021            |3225043739.22018780|
|817d18cd3c31e40e9bff0566baae7758|2020            |10117.46243062     |
|838a8fa992a4aa2fb5a0cf8b15b63755|2021            |340173600.19185339 |
|838a8fa992a4aa2fb5a0cf8b15b63755|2020            |4986.05430744      |
+--------------------------------+----------------+-------------------+



In [4]:
num_columns = len(samples.columns)
num_rows = samples.count()

print(f"Sample's shape: ({num_rows},{num_columns})")

Sample's shape: (21516918,8)


In [5]:
samples = samples.dropDuplicates().cache()
num_unique = samples.count()
if num_rows > num_unique:
    diff = num_rows - num_unique
    print(f"There a total of {diff} duplicate samples")

There a total of 11 duplicate samples


In [6]:
samples.show(5)

+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+------------------+----------------+
|         merchant_id|                 _id|          subsidiary|   transaction_date|      account_number|             user_id|transaction_amount|transaction_type|
+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+------------------+----------------+
|075d178871d8d4850...|0ccf1fe8dd3333e6e...|00015fd77a0f4d869...|2021-06-18 07:08:03|a79ff75955445c687...|ebc8d3ae01b8334f3...|      356.66730074|          DEBITO|
|075d178871d8d4850...|b86f53170772d5b15...|00015fd77a0f4d869...|2021-05-05 06:28:04|c790394d677140db4...|05ee34549ddebf8c2...|      350.72284572|          DEBITO|
|075d178871d8d4850...|cfe6f6fb72e5d5ad2...|00015fd77a0f4d869...|2021-07-09 05:48:27|c6a712474dc3e8c59...|de5f51b22e24411f3...|       35.66673007|         CREDITO|
|075d178871d8d4850...|

In [7]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F
from datetime import timedelta

In [8]:
window_spec = Window().partitionBy("user_id").orderBy("transaction_date")

In [9]:
samples = samples.withColumn("transaction_diff", F.lag("transaction_date").over(window_spec))

In [10]:
threshold = timedelta(hours=24)

In [11]:
samples = samples.withColumn("Fract", F.when((F.col("transaction_date") - F.col("transaction_diff") <= threshold) & (F.col("transaction_diff").isNotNull()), 1).otherwise(0)).cache()

In [12]:
samples = samples.where(samples['Fract'] == 1).cache()

In [13]:
samples.orderBy("user_id","transaction_date").show(10)

23/08/08 21:17:52 WARN MemoryStore: Not enough space to cache rdd_24_7 in memory! (computed 14.4 MiB so far)
23/08/08 21:17:52 WARN MemoryStore: Not enough space to cache rdd_24_3 in memory! (computed 14.5 MiB so far)
23/08/08 21:17:52 WARN MemoryStore: Not enough space to cache rdd_24_4 in memory! (computed 14.5 MiB so far)
23/08/08 21:17:55 WARN MemoryStore: Failed to reserve initial memory threshold of 1024.0 KiB for computing block rdd_24_9 in memory.
23/08/08 21:17:55 WARN MemoryStore: Not enough space to cache rdd_24_10 in memory! (computed 1302.4 KiB so far)
23/08/08 21:17:55 WARN MemoryStore: Not enough space to cache rdd_24_8 in memory! (computed 1301.6 KiB so far)
23/08/08 21:17:55 WARN MemoryStore: Not enough space to cache rdd_24_15 in memory! (computed 1303.0 KiB so far)
23/08/08 21:17:55 WARN MemoryStore: Not enough space to cache rdd_24_13 in memory! (computed 1301.4 KiB so far)
23/08/08 21:17:55 WARN MemoryStore: Not enough space to cache rdd_24_11 in memory! (computed 

+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+------------------+----------------+-------------------+-----+
|         merchant_id|                 _id|          subsidiary|   transaction_date|      account_number|             user_id|transaction_amount|transaction_type|   transaction_diff|Fract|
+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+------------------+----------------+-------------------+-----+
|817d18cd3c31e40e9...|ff835ae6aa80e7fff...|2ae700c733c02d781...|2021-09-20 18:21:00|6de86a7f937864c8a...|00000158a82e34eb0...|       23.77782004|          DEBITO|2021-09-20 18:19:55|    1|
|817d18cd3c31e40e9...|54134def4e7bfb19b...|4598e8c439f5b7d0a...|2021-11-23 14:48:07|10e6890a86c790e6a...|000004f4a6f3ac93f...|       47.55564009|          DEBITO|2021-11-23 14:38:39|    1|
|817d18cd3c31e40e9...|fb2051aab9031db96...|27364a641b7b

In [ ]:
spark.stop()